In [1]:
#apikey = X8vGMLWCXtOxkRhrgABZGj3baiaKApAn
#secret = VG1FpQufkmHA1F3n

#https://towardsdatascience.com/collecting-data-from-the-new-york-times-over-any-period-of-time-3e365504004
#https://developer.nytimes.com/docs/articlesearch-product/1/overview

In [9]:
import os
import json
import time
import requests
import datetime
import dateutil
import pandas as pd
from dateutil.relativedelta import relativedelta

### Getting data month by month (10 at a time) 

In [26]:
from dateutil.relativedelta import relativedelta
import pandas as pd
import datetime
end = datetime.date.today() + relativedelta(months=1)  # add one month to the end date
start = end - relativedelta(months=210)
months_in_range = [x.split(' ') for x in pd.date_range(start, end, freq='MS').strftime("%Y%m%d").tolist()]
months_in_range

#Date range: 20060201 to 20060301



[['20060201'],
 ['20060301'],
 ['20060401'],
 ['20060501'],
 ['20060601'],
 ['20060701'],
 ['20060801'],
 ['20060901'],
 ['20061001'],
 ['20061101'],
 ['20061201'],
 ['20070101'],
 ['20070201'],
 ['20070301'],
 ['20070401'],
 ['20070501'],
 ['20070601'],
 ['20070701'],
 ['20070801'],
 ['20070901'],
 ['20071001'],
 ['20071101'],
 ['20071201'],
 ['20080101'],
 ['20080201'],
 ['20080301'],
 ['20080401'],
 ['20080501'],
 ['20080601'],
 ['20080701'],
 ['20080801'],
 ['20080901'],
 ['20081001'],
 ['20081101'],
 ['20081201'],
 ['20090101'],
 ['20090201'],
 ['20090301'],
 ['20090401'],
 ['20090501'],
 ['20090601'],
 ['20090701'],
 ['20090801'],
 ['20090901'],
 ['20091001'],
 ['20091101'],
 ['20091201'],
 ['20100101'],
 ['20100201'],
 ['20100301'],
 ['20100401'],
 ['20100501'],
 ['20100601'],
 ['20100701'],
 ['20100801'],
 ['20100901'],
 ['20101001'],
 ['20101101'],
 ['20101201'],
 ['20110101'],
 ['20110201'],
 ['20110301'],
 ['20110401'],
 ['20110501'],
 ['20110601'],
 ['20110701'],
 ['2011080

### Getting Data by Pages (in the span of 24 months) 

In [28]:
def send_request(begin_date, end_date, page):
    '''Sends a request to the NYT Archive API for given date and page.'''
    
    url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json'
    api_key = 'X8vGMLWCXtOxkRhrgABZGj3baiaKApAn'
    query = 'Gold Price'
    sort = 'best'

    # Send the request to the API
    response = requests.get(url, params={'api-key': api_key, 'q': query, 'begin_date': begin_date, 'end_date': end_date, 'sort': sort, 'page': page})
    time.sleep(15)
    return response




def parse_response(response, start, end):
    '''Parses and returns response as pandas data frame.'''
    data = {'headline': [],  
        'date': [], 
        'doc_type': [],
        'material_type': [],
        'section': [],
        'keywords': []}
    
    articles = response.json()['response']['docs'] 
    for article in articles: # For each article, make sure it falls within our date range
        
        date = dateutil.parser.parse(article['pub_date']).date()
        data['date'].append(date)
        data['headline'].append(article['headline']['main']) 
        if 'section' in article:
            data['section'].append(article['section_name'])
        else:
            data['section'].append(None)
        data['doc_type'].append(article['document_type'])
        if 'type_of_material' in article: 
            data['material_type'].append(article['type_of_material'])
        else:
            data['material_type'].append(None)
        keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
        data['keywords'].append(keywords)
    return pd.DataFrame(data) 


def get_data(start, end):
    '''Sends and parses request/response to/from NYT Archive API for a given date range.'''
    total = 0
    print('Date range: ' + str(start) + ' to ' + str(end))
    if not os.path.exists('headlines_pages'):
        os.mkdir('headlines_pages')
    page = 0  # Set the initial page number to 0
    pagination_count = 0
    while pagination_count < 1:
        page += 1  # Increment the page number for each iteration
        response = send_request(start, end, page)
        if response.status_code != 200:
            print('Error:', response.json()['fault']['faultstring'])
            break
        df = parse_response(response, start, end)
        if len(df) == 0:
            break
        total += len(df)
        df.to_csv('headlines_pages/' + start + '-' + end + '_page' + str(page) + '.csv', index=False)
        print('Saving headlines/' + start + '-' + end + '_page' + str(page) + '.csv...')
        pagination_count += 1
    print('Number of articles collected: ' + str(total))

# Loop through each month pair
for i in range(len(months_in_range) - 1):
    start_month = months_in_range[i][0]
    end_month = months_in_range[i + 1][0]
    
    get_data(start_month, end_month)


Date range: 20060201 to 20060301
Saving headlines/20060201-20060301_page1.csv...
Number of articles collected: 10
Date range: 20060301 to 20060401
Saving headlines/20060301-20060401_page1.csv...
Number of articles collected: 10
Date range: 20060401 to 20060501
Saving headlines/20060401-20060501_page1.csv...
Number of articles collected: 10
Date range: 20060501 to 20060601
Saving headlines/20060501-20060601_page1.csv...
Number of articles collected: 10
Date range: 20060601 to 20060701
Saving headlines/20060601-20060701_page1.csv...
Number of articles collected: 10
Date range: 20060701 to 20060801
Saving headlines/20060701-20060801_page1.csv...
Number of articles collected: 10
Date range: 20060801 to 20060901
Saving headlines/20060801-20060901_page1.csv...
Number of articles collected: 10
Date range: 20060901 to 20061001
Saving headlines/20060901-20061001_page1.csv...
Number of articles collected: 10
Date range: 20061001 to 20061101
Saving headlines/20061001-20061101_page1.csv...
Number 

In [27]:
for i in range(len(months_in_range) - 1):
    start_month = months_in_range[i][0]
    end_month = months_in_range[i + 1][0]
    
    get_data(start_month, end_month)